<center>

<h1>Lab 2: Document Stores</h1>

<i>

Course: 23D020 Big Data Management for Data Science <br>

Author(s): Maria Simakova, Moritz Peist<br>

**Group: L2-T05**<br>

Programme: DSDM

<hr>

Note: This is a hands-on lab on Document Stores. We will be using one of the most popular document databases: MongoDB. We will practice how to import, create and model document databases, as well as how to query them.

</i>

</center>

<hr>

This notebook contains the comprehensive code and thus our solutions to the second lab. It can be run sequentially without errors, two execute all tasks in order.

**Note**, in order to use this notebook effectively, the user needs to create a `.env` file in the same directory as this notebook in order to connect to MongoDB. The `.env` file should have the following content structure:

```bash
CONNECTION_STRING=<connection_string>
DB_NAME=<database_name>
```

## Imports

In [161]:
# Libraries
import datetime
from pymongo import MongoClient
from faker import Faker
from time import perf_counter  # More accurate than time.time()
import random
from tqdm.notebook import tqdm
import pandas as pd
import dotenv

## B.1. Data Generation

In [162]:
class Lab2models:
    """
    A class for managing and operating on MongoDB database models.

    This class handles the creation of test data for different MongoDB data models,
    establishes connections to MongoDB, and provides methods for data generation
    and collection management.

    Parameters
    ----------
    num_companies : int
        Number of company documents to generate
    connection_string : str
        MongoDB connection string
    db_name : str
        Name of the MongoDB database to use

    Attributes
    ----------
    num_companies : int
        Number of companies to generate
    connection_string : str
        MongoDB connection string
    db_name : str
        MongoDB database name
    client : MongoClient
        MongoDB client instance
    db : Database
        MongoDB database instance
    """

    def __init__(
        self,
        num_companies,
        connection_string,
        db_name,
        faker=None,
        random_generator=None,
    ):
        # Set random seed for reproducibility
        Faker.seed(42)  # same people
        random.seed(42)  # for generating the same number of employees per company
        # Faker instance for generating fake data
        self.faker = faker or Faker(["es_ES"])
        self.random_generator = random_generator or random
        # Initialize the number of companies, employees will be generated accordingly
        self.num_companies = num_companies
        # MongoDB connection string and database name
        self.connection_string = connection_string
        self.db_name = db_name
        # Initialize MongoDB client and database
        self.client = None
        self.db = None

    def connect_to_mongo(self) -> None:
        """
        Connect to MongoDB using the provided connection string.

        Returns
        -------
        None
        """
        # Connect to MongoDB
        try:
            self.client = MongoClient(self.connection_string)
            self.db = self.client[self.db_name]
            print("✅ Connected to MongoDB\n")
        except Exception as e:
            print(f"❌ Error connecting to MongoDB: {e}\n")

    def list_collections(self) -> list:
        """
        List all collections in the MongoDB database.

        Returns
        -------
        list
            A list of collection names in the database
        """
        # Make sure we're connected first
        if self.db is None:
            self.connect_to_mongo()

        # List all collections in the database
        collections = self.db.list_collection_names()
        # print(f"Collections in {self.db_name}: {collections}\n")
        return collections

    def delete_collections(self) -> None:
        """
        Delete all model collections from the database.

        This method ensures we start with a clean database by removing
        all existing collections related to the models.

        Returns
        -------
        None
        """
        # Make sure we're connected first
        if self.db is None:
            self.connect_to_mongo()

        collections = self.db.list_collection_names()
        # Drop collections if they exist
        print(f"Deleting collections: {collections}")
        [self.db.drop_collection(collection) for collection in collections]

        # Now drop the collections
        # self.db.drop_collection("m1_people")
        # self.db.drop_collection("m1_companies")
        # self.db.drop_collection("m2_people")
        # self.db.drop_collection("m3_companies")

    ######################################################################## B1+2 ########################################################################
    def data_generator(self) -> tuple:
        """
        Generate test data for three different data models in MongoDB.

        This method orchestrates the entire data generation process by:
        1. Connecting to MongoDB and preparing collections
        2. Generating company and people data
        3. Storing data according to three different data models:
        - Model 1: Normalized model with people referencing companies
        - Model 2: Embedded model with companies embedded in people documents
        - Model 3: Embedded model with people embedded in company documents

        Returns
        -------
        tuple
            A tuple containing (number of companies, total number of employees)
        """
        # Initialize database and collections
        self._prepare_database()

        # Generate data
        company_data = self._generate_companies()
        company_ids, person_data = self._generate_people(company_data)
        total_employees = sum(len(persons) for _, persons in person_data.items())

        # Insert data into different models
        self._insert_model1_data(company_ids, person_data)
        self._insert_model2_data(company_data, person_data)
        self._insert_model3_data(company_data, person_data)

        return self.num_companies, total_employees

    def _prepare_database(self) -> tuple:
        """
        Connect to MongoDB and prepare collections for data insertion.

        This method connects to MongoDB, deletes existing collections,
        and creates new collections for all three data models.

        Returns
        -------
        tuple
            A tuple containing the four collection objects
        """
        # Connect to MongoDB
        self.connect_to_mongo()
        # Delete collection data if exists from all of the 3 models
        self.delete_collections()
        # Create and obtain collections
        m1_people = self.db.create_collection("m1_people")
        m1_companies = self.db.create_collection("m1_companies")
        m2_people = self.db.create_collection("m2_people")
        m3_companies = self.db.create_collection("m3_companies")

        self.collections = {
            "m1_people": m1_people,
            "m1_companies": m1_companies,
            "m2_people": m2_people,
            "m3_companies": m3_companies,
        }

        return self.collections

    def _generate_companies(self) -> list:
        """
        Generate company data for the specified number of companies.

        Uses Faker to create realistic company information including
        name, domain, email, URL, and VAT number.

        Returns
        -------
        list
            A list of dictionaries containing company data
        """
        # Set up faker
        fake = self.faker
        # Generate company data
        company_data = []
        for _ in range(self.num_companies):
            company_name = fake.company()
            domain = fake.domain_name()
            company = {
                "domain": domain,
                "email": f"info@{domain}",
                "name": company_name,
                "url": f"www.{domain}",
                "vatNumber": fake.bothify(text="??######"),
            }
            company_data.append(company)

        return company_data

    def _generate_people(self, company_data: list) -> tuple:
        """
        Generate employee data for each company.

        Creates a random number of employees for each company with
        realistic personal information including name, email, age, etc.

        Parameters
        ----------
        company_data : list
            List of company dictionaries generated by _generate_companies

        Returns
        -------
        tuple
            A tuple containing (company MongoDB IDs, dictionary mapping
            company indices to lists of person data)
        """
        fake = self.faker
        # Insert companies for Model 1 and get their IDs
        company_ids = (
            self.collections["m1_companies"].insert_many(company_data).inserted_ids
        )

        # Generate random number of employees per company
        employees_per_company = [
            self.random_generator.randint(25, 50) for _ in range(self.num_companies)
        ]

        # Organize people by company
        person_data = {company_idx: [] for company_idx in range(self.num_companies)}

        # For each company, generate the specific number of employees
        for company_idx in range(self.num_companies):
            for _ in range(employees_per_company[company_idx]):
                if fake.random_element(elements=(True, False)):
                    first_name = fake.first_name_male()
                    sex = "M"
                else:
                    first_name = fake.first_name_female()
                    sex = "F"

                last_name = fake.last_name()
                full_name = f"{first_name} {last_name}"
                birth_date = fake.date_of_birth(minimum_age=20, maximum_age=65)
                birth_datetime = datetime.datetime.combine(birth_date, datetime.time())
                age = (datetime.datetime.now().date() - birth_date).days // 365

                username_full = f"{first_name.lower()}.{last_name.lower()}".replace(
                    " ", ""
                )
                username_short = f"{first_name.lower()[0]}{last_name.lower()}".replace(
                    " ", ""
                )

                person = {
                    "age": age,
                    "companyEmail": f"{username_full}@{company_data[company_idx]['domain']}",
                    "dateOfBirth": birth_datetime,
                    "email": f"{username_short}@{fake.free_email_domain()}",
                    "firstName": first_name,
                    "fullName": full_name,
                    "sex": sex,
                }
                person_data[company_idx].append(person)

        return company_ids, person_data

    def _insert_model1_data(self, company_ids: list, person_data: dict) -> None:
        """
        Insert data using Model 1: Normalized model with people referencing companies.

        In this model, people documents contain a reference to their company.

        Parameters
        ----------
        company_ids : list
            List of MongoDB ObjectIDs for the companies
        person_data : dict
            Dictionary mapping company indices to lists of person data

        Returns
        -------
        None
        """
        m1_people = self.collections["m1_people"]
        for company_idx, persons in tqdm(
            person_data.items(), desc="Inserting data into Model 1: "
        ):
            for person in persons:
                m1_person = person.copy()
                m1_person["worksIn"] = company_ids[company_idx]  # Reference to company
                m1_people.insert_one(m1_person)

    def _insert_model2_data(self, company_data: list, person_data: dict) -> None:
        """
        Insert data using Model 2: Embedded model with companies embedded in people documents.

        In this model, each person document contains the full company document.

        Parameters
        ----------
        company_data : list
            List of company dictionaries
        person_data : dict
            Dictionary mapping company indices to lists of person data

        Returns
        -------
        None
        """
        m2_people = self.collections["m2_people"]
        for company_idx, persons in tqdm(
            person_data.items(), desc="Inserting data into Model 2: "
        ):
            for person in persons:
                m2_person = person.copy()
                m2_person["worksIn"] = company_data[
                    company_idx
                ]  # Embed company document
                m2_people.insert_one(m2_person)

    def _insert_model3_data(self, company_data: list, person_data: dict) -> None:
        """
        Insert data using Model 3: Embedded model with people embedded in company documents.

        In this model, each company document contains a list of all its employees.

        Parameters
        ----------
        company_data : list
            List of company dictionaries
        person_data : dict
            Dictionary mapping company indices to lists of person data

        Returns
        -------
        None
        """
        m3_companies = self.collections["m3_companies"]
        for company_idx, company in enumerate(
            tqdm(company_data, desc="Inserting data into Model 3: ")
        ):
            m3_company = company.copy()
            m3_company["staff"] = person_data.get(company_idx, [])
            m3_companies.insert_one(m3_company)

    ######################################################################## B4 ########################################################################
    def run_query(self, collection: str, pipeline: list, model_name: str) -> float:
        """
        Execute a MongoDB aggregation query and measure its performance.

        This method runs an aggregation pipeline against the specified collection,
        measures the execution time, and prints the results.

        Parameters
        ----------
        collection : str
            The name of the MongoDB collection to query
        pipeline : list
            MongoDB aggregation pipeline to execute
        model_name : str
            Name of the model being queried (for display purposes)

        Returns
        -------
        float
            Execution time of the query in seconds

        Notes
        -----
        The method prints the first 3 results and shows the total count of results
        along with the query execution time.
        """
        # Make sure we're connected first
        if self.db is None:
            self.connect_to_mongo()
        print(f"\n# {model_name}")
        start_time = perf_counter()
        results = list(self.db[collection].aggregate(pipeline))
        end_time = perf_counter()
        query_time = end_time - start_time
        # show the first 3 results
        for i, result in enumerate(results[:3]):
            print(result)
        if len(results) > 3:
            print(f"... and {len(results) - 3} other results")

        print(f"Time taken: {query_time:.4f} seconds")

        return query_time

    def run_update_query(
        self,
        collection: str,
        filter_query: dict,
        update_query: dict,
        array_filters: list,
        model_name: str,
    ) -> float:
        """
        Execute a MongoDB update operation and measure its performance.

        Parameters
        ----------
        collection_name : str
            The name of the MongoDB collection to update
        filter_query : dict
            MongoDB filter to select documents to update
        update_query : dict
            MongoDB update operation to apply
        model_name : str
            Name of the model being updated (for display purposes)

        Returns
        -------
        float
            Execution time of the update in seconds
        """
        # Make sure we're connected
        if self.db is None:
            self.connect_to_mongo()

        print(f"\n# {model_name}")

        # Start timing
        start_time = perf_counter()

        # Perform the update
        result = self.db[collection].update_many(
            filter_query, update_query, array_filters=array_filters
        )

        # End timing
        end_time = perf_counter()
        query_time = end_time - start_time

        print(f"Updated {result.modified_count} documents")
        print(f"Time taken: {query_time:.4f} seconds")

        return query_time

    def run_batch(
        self, models: list[dict[str, list, str]], is_update: bool = False
    ) -> dict[str, float]:
        """
        Run a batch of aggregation or update queries on MongoDB and measure performance.

        Parameters
        ----------
        models : list
            List of model configurations (dicts with name, pipeline, collection)
        is_update : bool
            Flag to indicate if the queries are update operations

        Returns
        -------
        dict
            Execution times for each query
        """
        # Connect if needed
        if self.db is None:
            self.connect_to_mongo()

        # Execute queries with dictionary comprehension
        if is_update:
            results = {
                model["name"]: self.run_update_query(
                    collection=model["collection"],
                    filter_query=model["filter"],
                    update_query=model["update"],
                    array_filters=model["array_filters"],
                    model_name=model["name"],
                )
                for model in models
            }
        else:
            results = {
                model["name"]: self.run_query(
                    collection=model["collection"],
                    pipeline=model["pipeline"],
                    model_name=model["name"],
                )
                for model in models
            }

        # Find fastest model in one operation
        fastest_model = min(results, key=results.get)
        fastest_time = results[fastest_model]

        # Print results
        print(f"\n{30 * '='} Model comparison: {30 * '='}")

        for name, t in results.items():
            print(f"{name}: {t:.4}s")

        print(f"The fastest model is {fastest_model} with: {fastest_time:.4}s")

        return results

    def close_connection(self):
        """
        Close the MongoDB connection.
        """
        if self.client:
            self.client.close()
            print("✅ MongoDB connection closed")
        else:
            print("❌ No MongoDB connection to close")

## Setup 

In [163]:
# Load environment variables from .env file
# The .env file should be in the same directory as this notebook, with the following content:
"""
CONNECTION_STRING=<connection_string>
DB_NAME=<database_name>
"""

dotenv.load_dotenv()

# Load MongoDB connection string from environment variables
connection_string = dotenv.get_key(
    ".env", "CONNECTION_STRING"
)  # Usually: "mongodb://localhost:27017/"

# Load MongoDB database name from environment variables
db_name = dotenv.get_key(".env", "DB_NAME")  # Some database name, e.g. "lab2"

# Setup parameters - shouldn't be hardcoded as per the instructions
# connection_string = "mongodb://localhost:27017/"
# db_name = "lab2"
num_companies = 50000

# Create an instance of the Lab2models class
lab = Lab2models(
    num_companies=num_companies, connection_string=connection_string, db_name=db_name
)

## B.2. Data Insertion

In [164]:
# Generate data
num_companies, actual_people = lab.data_generator()

# Print the number of documents in each collection
print(
    f"Actual number of documents: {num_companies} companies and {actual_people} people"
)

✅ Connected to MongoDB

Deleting collections: ['m2_people', 'm3_companies', 'm1_people', 'm1_companies']


Inserting data into Model 1:   0%|          | 0/500 [00:00<?, ?it/s]

Inserting data into Model 2:   0%|          | 0/500 [00:00<?, ?it/s]

Inserting data into Model 3:   0%|          | 0/500 [00:00<?, ?it/s]

Actual number of documents: 500 companies and 18523 people


## B.3. Queries

### Q1: For each person, retrieve their full name and their company’s name.

In [165]:
q1_1 = [
    {
        "$lookup": {
            "from": "m1_companies",
            "localField": "worksIn",
            "foreignField": "_id",
            "as": "companyName",
        }
    },
    {"$unwind": "$companyName"},
    {"$project": {"_id": 0, "fullName": 1, "companyName": "$companyName.name"}},
]

q1_2 = [{"$project": {"_id": 0, "fullName": 1, "companyName": "$worksIn.name"}}]

q1_3 = [
    {"$unwind": "$staff"},
    {"$project": {"_id": 0, "fullName": "$staff.fullName", "companyName": "$name"}},
]

In [166]:
queries_q1 = [
    {
        "name": "Model 1",
        "collection": "m1_people",
        "pipeline": q1_1,
    },
    {
        "name": "Model 2",
        "collection": "m2_people",
        "pipeline": q1_2,
    },
    {
        "name": "Model 3",
        "collection": "m3_companies",
        "pipeline": q1_3,
    },
]

In [167]:
q1_res = lab.run_batch(queries_q1)


# Model 1
{'fullName': 'Lola Reyes', 'companyName': 'Banca Privada OLMJ S.L.N.E'}
{'fullName': 'Adriana Palomino', 'companyName': 'Banca Privada OLMJ S.L.N.E'}
{'fullName': 'Nidia Saldaña', 'companyName': 'Banca Privada OLMJ S.L.N.E'}
... and 18520 other results
Time taken: 0.3162 seconds

# Model 2
{'fullName': 'Lola Reyes', 'companyName': 'Banca Privada OLMJ S.L.N.E'}
{'fullName': 'Adriana Palomino', 'companyName': 'Banca Privada OLMJ S.L.N.E'}
{'fullName': 'Nidia Saldaña', 'companyName': 'Banca Privada OLMJ S.L.N.E'}
... and 18520 other results
Time taken: 0.0261 seconds

# Model 3
{'fullName': 'Lola Reyes', 'companyName': 'Banca Privada OLMJ S.L.N.E'}
{'fullName': 'Adriana Palomino', 'companyName': 'Banca Privada OLMJ S.L.N.E'}
{'fullName': 'Nidia Saldaña', 'companyName': 'Banca Privada OLMJ S.L.N.E'}
... and 18520 other results
Time taken: 0.0209 seconds

============================== Model comparison: ==============================
Model 1: 0.3162s
Model 2: 0.02614s
Model 3: 0.

### Q2: For each company, retrieve its name and the number of employees.

In [168]:
q2_1 = [
    {"$group": {"_id": "$worksIn", "employeeCount": {"$sum": 1}}},
    {
        "$lookup": {
            "from": "m1_companies",
            "localField": "_id",
            "foreignField": "_id",
            "as": "companyName",
        }
    },
    {"$unwind": "$companyName"},
    {"$project": {"_id": 0, "companyName": "$companyName.name", "employeeCount": 1}},
]  # for the order of the output, first company then number of employees

q2_2 = [
    {
        "$group": {
            "_id": "$worksIn._id",
            "companyName": {"$first": "$worksIn.name"},
            "employeeCount": {"$sum": 1},
        }
    },
    {"$project": {"_id": 0, "companyName": 1, "employeeCount": 1}},
]

q2_3 = [
    {
        "$project": {
            "_id": 0,
            "companyName": "$name",
            "employeeCount": {"$size": "$staff"},
        }
    }
]

In [169]:
queries_q2 = [
    {
        "name": "Model 1",
        "collection": "m1_people",
        "pipeline": q2_1,
    },
    {
        "name": "Model 2",
        "collection": "m2_people",
        "pipeline": q2_2,
    },
    {
        "name": "Model 3",
        "collection": "m3_companies",
        "pipeline": q2_3,
    },
]

In [170]:
q2_res = lab.run_batch(queries_q2)


# Model 1
{'employeeCount': 33, 'companyName': 'Distribuciones Díez S.A.'}
{'employeeCount': 45, 'companyName': 'Tovar y Cisneros S.Coop.'}
{'employeeCount': 47, 'companyName': 'Transportes Roda y asociados S.L.'}
... and 497 other results
Time taken: 0.0158 seconds

# Model 2
{'companyName': 'Transportes Roda y asociados S.L.', 'employeeCount': 47}
{'companyName': 'Tovar y Cisneros S.Coop.', 'employeeCount': 45}
{'companyName': 'Distribuciones Díez S.A.', 'employeeCount': 33}
... and 497 other results
Time taken: 0.0099 seconds

# Model 3
{'companyName': 'Banca Privada OLMJ S.L.N.E', 'employeeCount': 25}
{'companyName': 'Grupo Rocamora S.Com.', 'employeeCount': 48}
{'companyName': 'Restauración CR S.Coop.', 'employeeCount': 33}
... and 497 other results
Time taken: 0.0033 seconds

============================== Model comparison: ==============================
Model 1: 0.01584s
Model 2: 0.009854s
Model 3: 0.003259s
The fastest model is Model 3 with: 0.003259s


### Q3: For each person born before 1988, update their age to “30”.

In [171]:
# Model 1 params
q3_1_filter = {"dateOfBirth": {"$lt": datetime.datetime(1988, 1, 1)}}
q3_1_update = {"$set": {"age": "30"}}
# Model 2 params
q3_2_filter = {"dateOfBirth": {"$lt": datetime.datetime(1988, 1, 1)}}
q3_2_update = {"$set": {"age": "30"}}
# Model 3 params -- To update ALL matching staff members and not only the first one, we need to use arrayFilters
q3_3_filter = {
    "staff": {"$elemMatch": {"dateOfBirth": {"$lt": datetime.datetime(1988, 1, 1)}}}
}
q3_3_update = {"$set": {"staff.$[elem].age": "30"}}
q3_3_array_filters = [{"elem.dateOfBirth": {"$lt": datetime.datetime(1988, 1, 1)}}]

In [172]:
queries_q3 = [
    {
        "name": "Model 1",
        "collection": "m1_people",
        "filter": q3_1_filter,
        "update": q3_1_update,
        "array_filters": None,
    },
    {
        "name": "Model 2",
        "collection": "m2_people",
        "filter": q3_2_filter,
        "update": q3_2_update,
        "array_filters": None,
    },
    {
        "name": "Model 3",
        "collection": "m3_companies",
        "filter": q3_3_filter,
        "update": q3_3_update,
        "array_filters": q3_3_array_filters,
    },
]

In [173]:
q3_res = lab.run_batch(queries_q3, is_update=True)


# Model 1
Updated 11463 documents
Time taken: 0.1001 seconds

# Model 2
Updated 11463 documents
Time taken: 0.0749 seconds

# Model 3
Updated 500 documents
Time taken: 0.0199 seconds

============================== Model comparison: ==============================
Model 1: 0.1001s
Model 2: 0.07486s
Model 3: 0.0199s
The fastest model is Model 3 with: 0.0199s


### Q4: For each company, update its name to include the word “Company”.

In [174]:
# Model 1 params
q4_1_filter = {}
q4_1_update = [{"$set": {"name": {"$concat": ["$name", " Company"]}}}]

# Model 2 params
q4_2_filter = {}
q4_2_update = [{"$set": {"worksIn.name": {"$concat": ["$worksIn.name", " Company"]}}}]

# Model 3 params
q4_3_filter = {}
q4_3_update = [{"$set": {"name": {"$concat": ["$name", " Company"]}}}]

In [175]:
queries_q4 = [
    {
        "name": "Model 1",
        "collection": "m1_companies",  
        "filter": q4_1_filter,
        "update": q4_1_update,
        "array_filters": None
    },
    {
        "name": "Model 2",
        "collection": "m2_people",
        "filter": q4_2_filter,
        "update": q4_2_update,
        "array_filters": None
    },
    {
        "name": "Model 3",
        "collection": "m3_companies",
        "filter": q4_3_filter,
        "update": q4_3_update,
        "array_filters": None
    }
]

In [176]:
res_q4 = lab.run_batch(queries_q4, is_update=True)


# Model 1
Updated 500 documents
Time taken: 0.0071 seconds

# Model 2
Updated 18523 documents
Time taken: 0.1429 seconds

# Model 3
Updated 500 documents
Time taken: 0.0132 seconds

============================== Model comparison: ==============================
Model 1: 0.007077s
Model 2: 0.1429s
Model 3: 0.01325s
The fastest model is Model 1 with: 0.007077s


In [177]:
# Finally close the connection
lab.close_connection()

✅ MongoDB connection closed


## C. Performance Comparison

The following code, is just small helper code to turn our model results automatically into LaTex format for automated synchronization with the submission document through Overleaf.

Furthermore, it highlights which model performed best for which task in green.

In [178]:
# Add all results to a DataFrame
results = pd.DataFrame.from_dict(
    {
        "M1": [
            q1_res["Model 1"],
            q2_res["Model 1"],
            q3_res["Model 1"],
            # res_q4["Model 1"],
        ],
        "M2": [
            q1_res["Model 2"],
            q2_res["Model 2"],
            q3_res["Model 2"],
            # res_q4["Model 2"],
        ],
        "M3": [
            q1_res["Model 3"],
            q2_res["Model 3"],
            q3_res["Model 3"],
            # res_q4["Model 3"],
        ],
    },
    orient="index",
    columns=["Q1", "Q2", "Q3"],  # , "Q4"],
).style.format(precision=4)

In [179]:
# Convert to LaTeX and write to file
with open(r"results/model_comparison.tex", "w") as f:
    f.write(
        results.to_latex(
            caption="Query Execution Times per Model (in seconds)",
            label="tab:model_comparison",
            position="H",
            position_float="centering",
            hrules=True,
        )
    )

In [180]:
results.highlight_min(color="green")

,Q1,Q2,Q3
M1,0.3162,0.0158,0.1001
M2,0.0261,0.0099,0.0749
M3,0.0209,0.0033,0.0199
